In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



df = pd.read_csv("../Data/2022_10_03_Daten_6Monate_export.csv", sep=";")
#df = df[pd.to_numeric(df['Zyklus'], errors='coerce').notnull()] #remove non numeric cycles
df.head()

/tmp/ipykernel_29851/1878940090.py:7: DtypeWarning: Columns (19,20,21,23,34,35,40,45,46,56,65,76,87,97,106,115,127,137,146,155,167,176,187,197,206,215,225,234,244,253,261,272,282,322,323,324,326,337,338,340,344,354,355,365,374,385,396,406,415,424,436,446,455,464,476,485,496,506,515,524,535,544,555,565,574,583,593,638,639,640,642,653,654,656,660,670,671,681,690,701,712,722,731,740,752,762,771,780,792,801,812,822,831,840,851,860,871,881,890,901,911) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Data/2022_10_03_Daten_6Monate_export.csv", sep=";")


,SapNummer,WA,Nutzen,Liefernutzen,PartNumber,Ident_WA_P_LN_Pr,Ident_WA_P,Ident_WA_P_LN,IdentifierLage,dt_34_Cleaner_Laser,...,Circuposit20_5_1054_1/6,Circuposit16_1_1054_1/6,Circuposit16_2_1054_1/6,Circuposit16_3_1054_1/6,Circuposit16_4_1054_1/6,Circuposit16_5_1054_1/6,1/6Rückätzung Mittelwert,2/5Rückätzung Mittelwert,Rückätzung1/6,Rückätzung2/5
0,436425,CTBL,1,0A,1,CTBL_01_0A_01,CTBL_01,CTBL_01_0A,16,0.548948,...,0.0,1519.0,1096.0,2848.0,8800.0,3758.0,7.7,7.4,7.7,7.4
1,436425,CTBL,1,0A,2,CTBL_01_0A_02,CTBL_01,CTBL_01_0A,16,0.548948,...,0.0,1519.0,1096.0,2848.0,8800.0,3758.0,7.7,7.4,7.7,7.4
2,436425,CTBL,1,0A,3,CTBL_01_0A_03,CTBL_01,CTBL_01_0A,0,0.548948,...,0.0,1519.0,1096.0,2848.0,8800.0,3758.0,7.7,7.4,7.7,7.4
3,436425,CTBL,1,0A,4,CTBL_01_0A_04,CTBL_01,CTBL_01_0A,0,0.548948,...,0.0,1519.0,1096.0,2848.0,8800.0,3758.0,7.7,7.4,7.7,7.4
4,436425,CTBL,1,0B,1,CTBL_01_0B_01,CTBL_01,CTBL_01_0B,0,0.548948,...,0.0,1519.0,1096.0,2848.0,8800.0,3758.0,7.7,7.4,7.7,7.4


# Process encoding

In [38]:
df_sel = df.iloc[:20].copy()
Plasma_cols = ["dt_34_Cleaner_Laser","dt_34_Plasma_Cleaner","dt_34_Galv_Plasma"]
Plasma_pos = [0,1,2]
Laser_cols = ["dt_34_Plasma_Laser","dt_25_Plasma_Laser"] 
Laser_pos = [3,3]
df_sel = df_sel[Plasma_cols + Laser_cols]
df_sel = df_sel.transpose()
df_sel["k"] = Plasma_pos + Laser_pos
df_sel = df_sel[["k",0]]
df_sel

,k,0
dt_34_Cleaner_Laser,0,0.548948
dt_34_Plasma_Cleaner,1,0.826970
dt_34_Galv_Plasma,2,4.303630
dt_34_Plasma_Laser,3,1.375917
dt_25_Plasma_Laser,3,0.460296


## Positional encoding from "Attention is all you need"

Modified code from [source]("https://machinelearningmastery.com/a-gentle-introduction-to-positional-encoding-in-transformer-models-part-1/")

In [57]:
def getPositionEncodingColumns(df,pos, d, n=10000):

    p_cols =  []
    for i in np.arange(int(d/2)):
        
        denominator = np.power(n, 2*i/d)
        col_even = f"P_{str(2 * i)}"
        col_odd = f"P_{str(2 * i + 1)}"
        
        df[col_even] = np.sin(df[pos]/denominator)
        df[col_odd] = np.cos(df[pos]/denominator)
        
        p_cols.extend([col_even,col_odd])
        
    return df,p_cols

def getTotalEncodingColumns(df,pos_columns,val_column):
    e_cols = []
    for i,col in enumerate(pos_columns):
        col_name = f"E_{str(i)}"
        df[col_name] = df[col] + df[val_column]
        
        e_cols.append(col_name)
    return df,e_cols

In [58]:
d = 4
df_sel_2,p_cols = getPositionEncodingColumns(df_sel,"k",d,n=100)
df_en, e_cols = getTotalEncodingColumns(df_sel_2,p_cols,0)
display(df_en)

,k,0,P_0,P_1,P_2,P_3,E_0,E_1,E_2,E_3
dt_34_Cleaner_Laser,0,0.548948,0.000000,1.000000,0.000000,1.000000,0.548948,1.548948,0.548948,1.548948
dt_34_Plasma_Cleaner,1,0.826970,0.841471,0.540302,0.099833,0.995004,1.668441,1.367272,0.926803,1.821974
dt_34_Galv_Plasma,2,4.303630,0.909297,-0.416147,0.198669,0.980067,5.212927,3.887483,4.502299,5.283697
dt_34_Plasma_Laser,3,1.375917,0.141120,-0.989992,0.295520,0.955336,1.517037,0.385925,1.671438,2.331254
dt_25_Plasma_Laser,3,0.460296,0.141120,-0.989992,0.295520,0.955336,0.601416,-0.529697,0.755816,1.415632


# Calculate Attention
[source](https://sebastianraschka.com/blog/2023/self-attention-from-scratch.html)

In [77]:
d_q, d_k, d_v = 7,7,9

W_q = np.random.rand(d_q,d)
W_k = np.random.rand(d_k,d)
W_v = np.random.rand(d_v,d)


np.array((df_en[e_cols])).T

q = pd.DataFrame(np.matmul(W_q,np.array(df_en[e_cols]).T).T)
k = pd.DataFrame(np.matmul(W_k,np.array(df_en[e_cols]).T).T)
v = pd.DataFrame(np.matmul(W_v,np.array(df_en[e_cols]).T).T)

display(q)
display(k)
display(v)

,0,1,2,3,4,5,6
0,2.025944,2.365530,2.001931,1.934178,0.545835,3.232504,2.059183
1,2.664949,3.041214,2.795415,2.165740,1.001536,4.029866,3.167974
2,8.817953,10.044620,9.735152,7.759515,3.056795,12.524412,10.659385
3,3.229716,3.586807,3.752870,1.935384,0.751201,3.701403,3.802384
4,1.566415,1.672273,1.930192,0.261483,0.169387,1.239024,1.816381


,0,1,2,3,4,5,6
0,2.402854,2.994879,2.193864,2.480704,1.885376,1.152961,2.060452
1,3.845823,4.279667,2.838259,2.914512,3.251428,1.390484,2.935906
2,11.994815,13.697399,8.906982,8.678316,11.405359,4.423551,8.990496
3,4.264291,4.408997,2.716943,2.794330,3.509983,1.697639,2.516406
4,2.105629,1.805118,0.984249,1.125629,1.306156,0.871857,0.780211


,0,1,2,3,4,5,6,7,8
0,1.721145,2.260782,2.352933,2.480296,1.775228,2.957497,1.815858,2.320902,3.709478
1,2.137558,2.905616,2.999374,2.846676,2.967091,4.294519,3.196793,3.551687,5.079913
2,7.878646,9.100520,9.801709,8.522083,10.417577,13.891896,10.654654,11.964809,16.180840
3,2.333551,3.037909,2.723417,2.803896,3.789840,4.483363,3.521837,3.197355,4.895186
4,0.702335,1.304287,0.747225,1.162471,1.852273,1.840976,1.541517,0.811357,1.757106


In [90]:
omega = pd.DataFrame(np.dot(np.array(q),np.array(k).T))
display(omega)

,0,1,2,3,4
0,30.141512,41.549344,130.356669,42.497928,17.821336
1,40.078875,55.671251,175.046820,56.748130,23.583663
2,134.043846,185.794903,583.504613,188.836074,78.243926
3,45.055488,62.816237,197.218778,63.679794,26.322418
4,19.145834,27.027728,84.899049,27.296302,11.229720


In [91]:
from scipy.special import softmax

attention_weights = pd.DataFrame(softmax(np.array(omega) / d_k**0.5))
display(attention_weights)

,0,1,2,3,4
0,1.467633e-91,1.094432e-89,4.137362e-75,1.566379e-89,1.394077e-93
1,6.277902e-90,2.276513e-87,8.964317e-68,3.420090e-87,1.230723e-92
2,1.667115e-74,5.209456e-66,1.000000e+00,1.644350e-65,1.154853e-83
3,4.118272e-89,3.389256e-86,3.908384e-64,4.697377e-86,3.465143e-92
4,2.299853e-93,4.523726e-92,1.428734e-82,5.007053e-92,1.154215e-94


In [95]:
attention = np.matmul(np.array(attention_weights),np.array(v))
display(pd.DataFrame(attention))

,0,1,2,3,4,5,6,7,8
0,3.259681e-74,3.765215e-74,4.055322e-74,3.525894e-74,4.310129e-74,5.747580e-74,4.408216e-74,4.950274e-74,6.694599e-74
1,7.062668e-67,8.157995e-67,8.786563e-67,7.639465e-67,9.338647e-67,1.245314e-66,9.551170e-67,1.072563e-66,1.450502e-66
2,7.878646e+00,9.100520e+00,9.801709e+00,8.522083e+00,1.041758e+01,1.389190e+01,1.065465e+01,1.196481e+01,1.618084e+01
3,3.079277e-63,3.556833e-63,3.830884e-63,3.330757e-63,4.071589e-63,5.429486e-63,4.164248e-63,4.676307e-63,6.324093e-63
4,1.125649e-81,1.300222e-81,1.400403e-81,1.217579e-81,1.488394e-81,1.984782e-81,1.522266e-81,1.709452e-81,2.311811e-81
